In [ ]:
import json

import pandas as pd
from google_play_scraper import app


def get_game_list():
    with open("data/gameList.json", encoding='utf-8') as f:
        return json.load(f)


def get_country():
    with open('data/currencyRate.json', encoding='utf-8') as f:
        return json.load(f)


def get_game_info(name, country):
    return app(name, lang='en', country=country)

In [ ]:
def get_game_info_to_df(country):
    games = get_game_list()
    result = pd.DataFrame()
    for game in games:
        name = game["name"]
        gameid = game["packageId"]
        id = game["id"]
        country = country
        try:
            price = get_game_info(gameid, country)['inAppProductPrice']
        except Exception as e:
            print(f"{name} is not found due to {e}")
        price_df = pd.DataFrame([[name, country, price]], columns = ['遊戲名稱', '國家', '價格'])
        price_df['價格'] = price_df['價格'].str.split('-').str[-1].str.split(' ').str[1]
        result = pd.concat([result, price_df])
    return result

In [ ]:
countries = get_country()
for country in countries:
    CurrencyName = country['currencyName']
    Country = country['countryName']
    result = get_game_info_to_df(CurrencyName)
    result.to_csv(f"output/{Country}_info.csv", encoding="utf-8", index = None)
    break

In [ ]:
import pandas as pd

currency_data = pd.read_excel('output/即時匯率.xlsx')
currency_data['幣值簡稱'] = currency_data['幣值'].str[:2].str.upper()
price_data = pd.read_csv('output/約旦_info.csv')
price_data['幣值簡稱'] = price_data['國家'].str[:2].str.upper()

In [ ]:
currency_data = currency_data.rename(columns={'Visa': 'Visa匯率', 'Master': 'Master匯率', 'JCB': 'JCB匯率'})
currency_data

In [ ]:
result = pd.merge(currency_data, price_data, left_on = '幣值簡稱', right_on = '幣值簡稱', how = 'right')
result = result.drop(['國家_y', '幣值簡稱', '試算結果', '試算結果.1', '試算結果.2'], axis = 1)
result = result.rename(columns = {'國家_x': '國家'})
result = result[['國家', '幣值', '金額', 'VISA', '匯率', 'Master', '匯率.1', 'JCB', '匯率.2', '遊戲名稱', '價格']]
result = result.rename(columns = {'匯率': 'VISA匯率', '匯率.1': 'Master匯率', '匯率.2': 'JCB匯率'})
result = result.fillna(0)
result['價格'] = result['價格'].replace(r'[^\d.]', '', regex=True)
result.to_csv('test.csv', index = None)

In [ ]:
data = pd.read_csv('test.csv')
data['VISA 價格'] = data['VISA匯率'] * data['價格']
data['Master 價格'] = data['Master匯率'] * data['價格']
data['JCB 價格'] = data['JCB匯率'] * data['價格']
data

In [ ]:
price_data